# imports

In [1]:
import thunderfish.dataloader as dl
import thunderfish.pulsetracker as pt
import thunderfish.eventdetection as ed
import matplotlib.pyplot as plt
import numpy as np
import os

# parameters and functions


In [2]:
def plot_peaks(x,y,peaksx,peaksy,c='k'):
    plt.figure(figsize=(10,5))
    plt.plot(x,y,zorder=-1)
    plt.scatter(starttime + peaksx/data.samplerate,peaksy,c=c)
    plt.xlabel('time [s]')
    plt.ylabel('signal')
    
def plot_events_on_data(peaks, data):
    
    """
        plots the detected events onto the data timeseries. If the events are classified, the classes are plotted in different colors and the class -1 (not belonging to a cluster) is plotted in black
    """
    plt.plot(range(len(data)),data, color = 'black')
    if len(peaks)>3:
        classlist =  np.array(peaks[3],dtype=np.int)
        if len(peaks) > 4:
            classlist = np.array(peaks[4],dtype=np.int)
        cmap = plt.get_cmap('jet')
        colors =cmap(np.linspace(0, 1.0, 3000)) #len(np.unique(classlist))))
        np.random.seed(3)
        np.random.shuffle(colors)
        colors = [colors[cl] for cl in np.unique(classlist)]
        for cl, color in zip(np.unique(classlist), colors):
            if min(classlist) == 0 and cl == 0:
                color = 'black'
            if cl == -1:
                color = 'black'
            
            peaksofclass = peaks[:,classlist == cl]
            plt.plot(peaksofclass[0],peaksofclass[1], '.', color = color,   ms =20, label=cl)
    else:
        plt.scatter(peaks[0],peaks[1], color = 'red')
    #plt.legend()

# load data

In [3]:
# create list of filepaths..
text_file = open("leticia_filenames_sorted.txt", "r")
lines = text_file.read().split('\n')
i = 4

In [4]:
filepath = '/home/dexter/' + lines[i][:-4] + '/' + lines[i]

In [5]:
filepath

'/home/dexter/31105L01F14/31105L01F14.WAV'

In [6]:
starttime = 0
endtime = 1000 #in seconds

with dl.open_data(filepath, -1, 1.0) as data:
    dt = 1/data.samplerate
    # do something with the content of the file:
    x = np.arange(starttime,endtime,dt)
    y = data[starttime*data.samplerate:endtime*data.samplerate,0]

# online clustering step by step
## 1. EOD detection
Create a data structures with the locations and amplitudes of the EODs

In [7]:
from scipy.interpolate import interp1d
from scipy.signal import correlate
from scipy.signal import argrelextrema

# parameters for the analysis
thresh = 0.1 # minimal threshold for peakdetection
peakwidth = 25 # width of a peak and minimal distance between two EODs

In [8]:
pk, tr = ed.detect_peaks(y, thresh)
peaks = pt.makeeventlist(pk,tr,y,peakwidth)
peakindices, peakx, peakh = pt.discardnearbyevents(peaks[0],peaks[1],peakwidth)
peaks = np.transpose(peaks[:,peakindices])

## 2. Online clustering
create object

In [9]:
class cluster_object(object):
    def __init__(self,label,mean,features,peaknum):
        self.label = label
        self.mean = mean
        # this should be a numpy array with at least 100 peaks?
        self.features = features
        self.idxs = np.zeros(peaknum)
        self.y = np.zeros(peaknum)
        
    def predict_next_idx(self):
        
        np.nonzero(cluster.idxs)[0][-1]
        
        return np.max(self.idxs) + self.get_isi()
    
    def get_isi(self):
        return np.mean(np.diff(self.idxs[np.nonzero(self.idxs)[0]][-4:]))

In [10]:
def align(eod,cluster_mean,width):
    
    # don't just take max but take a peak of the cc
    # this is to prevent shifting to the next peak in the sequence.
    
    cc = correlate(eod,cluster_mean,'same')
    local_max_idx = argrelextrema(cc[width:-width], np.greater)[0] + width
    diff = np.abs(local_max_idx - len(eod)/2)
    if diff.size > 0:
        offset = local_max_idx[np.argmin(diff)]
    else:
        offset = int(len(eod)/2)
    return extract(eod,offset,width)
    
def extract(eod,offset,width):
    return eod[offset-width:offset+width]

def normalize(x):
    #return x - np.min(x)
    return (((x-np.min(x))/(np.max(x)-np.min(x))) - 0.5)*2

def gen_cheb(n,x):
    return(np.cos(n*np.arccos(x)))

#cmap = plt.get_cmap('jet')    
colorss = ['k','b','r']#cmap(np.linspace(0, 1.0, 3000)) #len(np.unique(classlist))))
#np.random.seed(3)
#np.random.shuffle(colors)

In [11]:
%matplotlib qt

In [12]:
clusters = []
deleted_clusters = []

maxlabel = 1
labels = np.zeros(len(peaks))

# parameters
cutwidth = peakwidth
alignwidth = 150
int_met = 'quadratic'
eta = 0.1
error_threshold = 0.1 #different thresholds are needed for different problems
npol = 15
nfeat = 7

#decide on the plotting
plot_steps = False
plot_means = False
plot_one_peak_step = False

if plot_means == True:
    f, (ax1, ax2) = plt.subplots(1, 2,figsize=(10,5))

for i,p in enumerate(peaks):
    
    if plot_steps == True:
        f, (ax1, ax2) = plt.subplots(1, 2,figsize=(10,5))
        #f, ax1 = plt.subplots(1, 1,figsize=(5,5))
        ax1.axis("off")
        ax2.axis("off")
    
    # cut along peak to get the EOD shape
    p_idx = int(p[0])
    p_hight = p[2]
    p_y = p[1]
    
    eod = y[p_idx-cutwidth:p_idx+cutwidth]
    
    #interpolate --> needed to properly align peaks
    eod_func = interp1d(range(len(eod)), eod, int_met)
    eod_ip = eod_func(np.arange(0,len(eod)-1,0.1))
    
    # align peaks with aligning peak
    if len(clusters) > 0:
        
        current_clusters = []
        
        # I only want to use recent clusters, 
        # but somehow I also need to save all clusters for later
        for c, cluster in enumerate(clusters):
            # delete clusters that are too far away
            if p_idx - np.max(cluster.idxs) < 2000000: #and np.count_nonzero(labels == cluster.label) < 3:
                current_clusters.append(cluster)
            else:
                deleted_clusters.append(cluster)
                
        clusters = current_clusters
 
        c_feats = np.zeros((len(clusters),nfeat+1))
        
        # ============== start analysis =================
        for c,cluster in enumerate(clusters):
            c_feats[c] = cluster.features
            
            if plot_steps == True:
                ax1.plot(cluster.mean,c=colorss[int(cluster.label)],label='fish %i'%cluster.label,linewidth=3.0)
                ax1.title.set_text('Normalized EODs')
                ax2.plot(cluster.features,c=colorss[int(cluster.label)],label='fish %i'%cluster.label,linewidth=3.0)
                ax2.title.set_text('Features')
        aligned_eod = align(eod_ip, align_peak, alignwidth)
        
        # normalize
        mean = normalize(aligned_eod)
        
        # extract features
        cheb = np.polynomial.chebyshev.Chebyshev.fit(np.linspace(-1,1,
            len(mean)), mean, npol)
        
        #append peak hight to features
        features = np.append(cheb.coef[:nfeat], p_hight/2)
        
        if plot_one_peak_step == True:
            
            chebfit = np.zeros(len(mean))
            t = np.linspace(0,len(mean)*dt*0.1*1000,len(mean))
            
            for n in range(npol+1):
                f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4,figsize=(15,5))
                
                ax1.plot(t,mean,c='r',linewidth=3.0)
                ax1.title.set_text('1. Normalized Peak')
                
                current_cheb = gen_cheb(n,np.linspace(-1,1,len(mean)))
                chebfit = chebfit + cheb.coef[n]*current_cheb
                
                ax1.plot(t,chebfit, c='k',linestyle='--',linewidth=2.0)
                ax1.set_xlabel('Time [ms]')

                ax2.title.set_text('2. Fitting Chebyshev polinomial')
                ax2.plot(t,current_cheb,c='k',linewidth=3.0)
                ax2.set_xlabel("Time [ms]")
                
                ax3.plot(cheb.coef,c='k')
                ax3.plot(n, cheb.coef[n], '.',c='k', ms=20)
                ax3.title.set_text('3. Chebyshev coefficients')
                ax3.set_xlabel('Coefficient #')
                
                ax4.axis('off')
                f.savefig('a%i.png'%n)
            
            f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4,figsize=(15,5))
            ax1.plot(t,mean,c='r',linewidth=3.0)
            ax1.title.set_text('1. Normalized Peak')
                
            ax1.plot(t,chebfit, c='k',linestyle='--',linewidth=2.0)
            ax1.set_xlabel("Time [ms]")

            ax2.title.set_text('2. Fitting Chebyshev polinomial')
            ax2.plot(t,current_cheb,c='k',linewidth=3.0)
            ax2.set_xlabel("Time [ms]")
            
            ax3.plot(cheb.coef,c='k')
            ax3.plot(npol, cheb.coef[-1], '.',c='k', ms=20)
            ax3.title.set_text('3. Chebyshev coefficients')
            ax3.set_xlabel('Coefficient #')
            
            ax4.axis('on')
            ax4.plot(features,c='r',linewidth=3.0,alpha=0.5)
            ax3.plot(features[:-1],c='r',linewidth=3.0,alpha=0.5)
            ax4.title.set_text('4. Peak features')
            ax4.set_xlabel('Feature #')
            f.savefig('a%i.png'%npol)
                
            break
        
        # compute difference in feature vectors
        cluster_errors = np.linalg.norm(c_feats - features, axis=1)
        
        if plot_steps == True:
            #ax1.plot(mean,c='k',linestyle='--',zorder=3,label='new peak',linewidth=3.0)
            #ax2.plot(features,c='k',linestyle='--',zorder=3,label='new peak',linewidth=3.0)
            ax1.legend()
            ax2.legend()

        if np.min(cluster_errors) < error_threshold:
            # assign eod to the class with the lowest cluster_error
            labels[i] = clusters[np.argmin(cluster_errors)].label

            #adapt this cluster with learning rate eta
            #clusters[np.argmin(cluster_errors)].mean = (1-eta)*  \
            #clusters[np.argmin(cluster_errors)].mean +       \
            #    eta*mean[np.argmin(cluster_errors)]

            clusters[np.argmin(cluster_errors)].features = (1-eta)*  \
                clusters[np.argmin(cluster_errors)].features +       \
                eta*features        

            clusters[np.argmin(cluster_errors)].idxs[
                (clusters[np.argmin(cluster_errors)
                         ].idxs==0).argmax(axis=0)] = p_idx
            
            clusters[np.argmin(cluster_errors)].y[
                (clusters[np.argmin(cluster_errors)
                         ].y==0).argmax(axis=0)] = p_y
            
        else:
            # check for collisions
            # check out predicted next peak for all clusters
            # if this predicted peak is in the current area
            # for 2 or more cluster, assign this peak to all
            # of those cluster.
            idx_pred = np.zeros(len(clusters))
            last_idx = np.zeros(len(clusters))
            for c,cluster in enumerate(clusters):
                idx_pred[c] = cluster.predict_next_idx()
                last_idx[c] = np.max(cluster.idxs)
            pred_error = np.abs(idx_pred - p_idx)
            
            if np.count_nonzero(pred_error<peakwidth*4) >= 2:
                # assign this peak to both clusters but do not 
                # adapt the cluster features
                labels[i] = -1
                for j in np.nonzero(pred_error<peakwidth*4)[0]:
                    clusters[j].idxs[
                        (clusters[j].idxs==0).argmax(axis=0)] = p_idx
                    clusters[j].y[
                        (clusters[j].y==0).argmax(axis=0)] = p_y
            
            else:    
                # create new cluster class
                labels[i] = maxlabel

                maxlabel = maxlabel + 1
                new_cluster = cluster_object(labels[i],mean,
                                         features,len(peaks))
                new_cluster.idxs[0] = p_idx
                new_cluster.y[0] = p_y

                clusters.append(new_cluster)

            if plot_means == True:
                    ax1.plot(mean,c=colors[int(labels[i])])
                    ax2.plot(features,c=colors[int(labels[i])])
    else:
        # create new cluster class
        labels[i] = maxlabel
        maxlabel = maxlabel + 1
        
        # take middle of snip
        mid = int(len(eod_ip)/2)
        eod_ip = extract(eod_ip,mid,alignwidth)
        
        # normalize
        eod_ip = normalize(eod_ip)
        
        # set this as the standard peak for alignment
        align_peak = eod_ip
        
        # extract features
        cheb = np.polynomial.chebyshev.chebfit(np.linspace(-1,1,
            len(eod_ip)), eod_ip, npol)
        
        #features = cheb.coef
        features = np.append(cheb[:nfeat], p_hight/2)
        #features = cheb
        
        new_cluster = cluster_object(labels[i],eod_ip,features,len(peaks))
        new_cluster.idxs[0] = p_idx
        new_cluster.y[0] = p_y
        clusters.append(new_cluster)
    
    # save eods every n steps. before saving delete the short classes.
    # save the peaks of the current buffered part to a numpy-memmap on the disk
    # ==> sth like this: save_EOD_events_to_npmmp(thisblock_eods,eods_len,idx==startblock,datasavepath,mmpname)

/home/liz/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/liz/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/liz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:179: RuntimeWarning: invalid value encountered in less
/home/liz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:183: RuntimeWarning: invalid value encountered in less


In [13]:
peaks = np.append(np.transpose(peaks),[labels], axis = 0)

In [14]:
peaks_ns = pt.discard_short_classes(peaks, 3)

# set peaks that don't have many instances to the noise class.

In [15]:
peaks_ns.shape

(4, 30276)

In [16]:
%matplotlib qt
plt.axis("off")
for c in clusters:
    if np.count_nonzero(c.idxs) > 10:
        plt.plot(c.mean,c=colorss[int(c.label)])

IndexError: list index out of range

In [ ]:
for c in clusters:
    if np.count_nonzero(c.idxs) > 10:
        plt.plot(c.features,c=colors[int(c.label)])

In [17]:
%matplotlib qt
plt.plot(y,c='k',alpha=0.3)

for cluster in clusters:
    cx = cluster.idxs[np.nonzero(cluster.idxs)]
    cy = cluster.y[np.nonzero(cluster.y)]
    if len(cx) > 10:
        plt.plot(cx,cy,'.',alpha=0.5,ms=20,label=cluster.label)
    
for cluster in deleted_clusters:
    cx = cluster.idxs[np.nonzero(cluster.idxs)]
    cy = cluster.y[np.nonzero(cluster.y)]
    if len(cx) > 10:
        plt.plot(cx,cy,'.',alpha=0.5,ms=20,label=cluster.label)

In [ ]:
len(clusters)

In [ ]:
print(peaks[0])

In [ ]:
len(clusters)